# C. Live Streaming Twitter Alerts

The streaming API code heavily borrowed from 

In [ ]:
import numpy as np
import pandas as pd

import tweepy
from tweepy import StreamListener, Stream
import json 
from textblob import TextBlob
from geopy import geocoders 

%matplotlib notebook

from matplotlib import pyplot as plt
from matplotlib.animation import FuncAnimation
import matplotlib.animation as animation
#import tmp102
import datetime as dt
from threading import Thread
import time

# input your Twitter API credentials
consumer_key = "CONSUMER KEY"
consumer_secret = "CONSUMER SECRET"
access_token = "ACCESS TOKEN"
access_secret = "ACCESS SECRET"
 
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

#alert types to pick for filter
alert_fire = [' fire ', ' wildfire ', ' blaze ', ' campfire ', ' smoke ']
alert_flood = [' flood ', ' flooding ', ' flooded ', ' surge ', ' torrent ']
alert_storm = [' storm ', ' tornado ', ' wind ', ' snow ', ' hale ', ' rain ', ' deluge ', ' hurricane ', ' avalanche ', ' ice ']
alert_shooting = [' shooting ', ' shooter ', ' shots fired ', ' bullet ', ' gunshot ', ' murder ', ' driveby ']
alert_any = alert_fire + alert_flood + alert_storm + alert_shooting

# additional filter for urgency - only works on top of other filters
urgent = [' help ', ' HELP ', ' sos ', ' SOS ', ' police ', ' emt ', ' ems ', ' emergency ', 
          ' assistance ', ' assist ', ' 911 ', ' relief ', ' warning ', ' danger ', ' crisis ']

# test set for alerts
shutdown_test = [' shutdown ', ' government ', ' federal ', ' tsa ', 'Trump', ' democrats ',
                'Pelosi', ' republicans ', 'Coulter']

# calculating coordinates of a location
def bounding_coord(city_state, radius=15):
    gn = geocoders.GeoNames(username = 'katerdowdy') 
    for _ in range(3): # setting number of retries
        try:
            coords = gn.geocode(city_state, exactly_one=False)[0]
            lat = coords.latitude
            long = coords.longitude
        except:
            print("Location coordinates not found, try again!")
        
    degree_change = (radius / 69)
    sw_lat = lat - (degree_change / 2)
    sw_long = long - (degree_change / 2)
    ne_lat = lat + (degree_change / 2)
    ne_long = long + (degree_change / 2)
    
    return sw_long, sw_lat, ne_long, ne_lat

# converting unix to Eastern time
def convert_eastern(unix):
    zulu = pd.to_datetime(unix, unit = 'ms')
    eastern = zulu - pd.Timedelta(hours=5)
    round_time = pd.DatetimeIndex.round(eastern, freq = 's')
    return round_time
